In [1]:
from __future__ import print_function, division
import sys,os
# line 4 and line 5 below are for development purposes and can be removed
qspin_path = os.path.join(os.getcwd(),"../../")
sys.path.insert(0,qspin_path)
from scipy.linalg import expm
from quspin.operators import hamiltonian, commutator, exp_op # Hamiltonians and operators
from quspin.basis import tensor_basis,spinful_fermion_basis_1d,boson_basis_1d # bases
from quspin.tools.measurements import obs_vs_time # calculating dynamics
from quspin.tools.Floquet import Floquet_t_vec # period-spaced time vector
import numpy as np # general math functions
import matplotlib.pyplot as plt # plotting library
#
##### setting up parameters for simulation
# physical parameters
l = 1
L_f = 3 # system size f
L_b = L_f-1 # system size b
Nf, Nb = 2, l # number of fermions, bosons
N = Nf + Nb # total number of particles
tb, tf = 0.1, 0.1 # boson, fermon hopping strength
U = 1.0  # n(n-1) interaction strength
V = 1.0
mu_b, mu_f = -0.5, -0.5 # b, f chemical potential 
# define time-dependent perturbation
A = 0
Omega = 1.0
def drive(t,Omega):
    return np.sin(Omega*t)
drive_args=[Omega]
#
###### create the basis
# build the two bases to tensor together to a bose-fermi mixture
basis_b1=boson_basis_1d(L_b,Nb=[i for i in range(2*l*L_b+1)],sps=2*l+1) # boson basis
basis_b2=boson_basis_1d(L_b,Nb=[i for i in range(2*l*L_b+1)],sps=2*l+1) # boson basis
basis_f=spinful_fermion_basis_1d(L_f,Nf = [(0,0),(0,1),(1,0),(1,1),(2,0),(0,2),(1,2),(2,1),(2,2),(0,3),(3,0),(3,1),(1,3),(2,3),(3,2),(3,3)],double_occupancy=True)
basis=tensor_basis(basis_b1,basis_b2,basis_f) # BFM
#
print(basis_b1)
print(basis_f)
##### create model
# define site-coupling lists
hop_b = [[-tb,i,(i+1)] for i in range(L_b-1)] # b hopping
int_list_bb = [[U/2.0,i,i] for i in range(L_b)] # bb onsite interaction
int_list_bb_lin = [[-U/2.0,i] for i in range(L_b)] # bb interaction, linear term
# 
hop_f_right = [[-tf,i,(i+1)] for i in range(L_f-1)] # f hopping right
hop_f_left = [[tf,i,(i+1)] for i in range(L_f-1)] # f hopping left
drive_f = [[A*(-1.0)**i,i] for i in range(L_f)] # density staggered drive
#
chem_f = [[mu_f,i] for i in range(L_f)] #chemical potential for fermions
chem_b = [[mu_b,i] for i in range(L_b)] #chemical potential for bosons
#
int_list_bf = [[V,i,j] for i in range(L_b) for j in range(L_f)] # bf onsite interaction
int_list_bf_m = [[-V,i,j] for i in range(L_b) for j in range(L_f)] # bf onsite interaction

#int_list_bf_offsite = [[V,i,i+1] for i in range(L_b)] # bf onsite interaction

penalty_quad = [[1000.,i,i] for i in range(L_b)] #penalty in order to stay in the sector n_b + n_d = 2l
penalty_lin = [[-4000.*l,i] for i in range(L_b)]
penalty_const = [[1000.*4*l**2,i] for i in range(L_b)]
penalty_quad_2 = [[2000.,i,i] for i in range(L_b)]




# create resource Hamiltonian
bosons_hopping = [	
            ["+-|||", hop_b], # bosons hop left
            ["|+-||", hop_b], # bosons hop left

            ["-+|||", hop_b], # bosons hop right
            ["|-+||", hop_b], # bosons hop right
]

bosons_on_site_int = [
            ["n|||", int_list_bb_lin], # bb onsite interaction
            ["|n||", int_list_bb_lin], # bb onsite interaction

            ["nn|||", int_list_bb], # bb onsite interaction
            ["|nn||", int_list_bb], # bb onsite interaction
]
            #
fermions_hopping = [
            ["|||+-", hop_f_left], # fermions hop left
            ["||+-|", hop_f_left], # fermions hop left

            ["|||-+", hop_f_right], # fermions hop right
            ["||-+|", hop_f_right], # fermions hop right
]
            #
boson_fermion_int_b1 = [

            ["n||n|", int_list_bf], # bf onsite interaction

]

boson_fermion_int_b2 = [
            ["n|||n", int_list_bf_m], # bf onsite interaction

]

boson_fermion_int_d1 = [

            ["|n|n|", int_list_bf], # bf onsite interaction

]

boson_fermion_int_d2 = [

            ["|n||n", int_list_bf_m], # bf onsite interaction
]

            #
chem_potential_f = [
            ["|||n", chem_f], #chemical potential fermions
            ["||n|", chem_f], #chemical potential fermions
]
chem_potential_b = [
            ["n|||", chem_b],  #chemical potential bosons 
            ["|n||", chem_b],  #chemical potential bosons    

            ]
dynamic = [["|n",drive_f,drive,drive_args]] # drive couples to fermions only

############################################## target  hamiltonian
a = 0.1
e = 1.0
m = 0.3

bosonic_part_LGT = [[a*e**2/4.,i,i,i,i] for i in range(L_b)] # b^dagger*b^dagger*b*b
mass_f = [[m+1.0/a,i,i] for i in range(L_f)] # f mass term
m_mass_f = [[-m-1.0/a,i,i] for i in range(L_f)] # f mass term

interaction_part_LGT = [[1.0/(a*np.sqrt(l*(l+1))),i,i,i,i+1] for i in range(L_b)] #boson-fermion interaction
m_interaction_part_LGT = [[-1.0/(a*np.sqrt(l*(l+1))),i,i,i,i+1] for i in range(L_b)] #boson-fermion interaction

coherent_b = [[1.,i] for i in range(L_b)]
coherent_f = [[1.,i] for i in range(L_f)]


#
static_coh = [
    
    ["+|||",coherent_b],
    ["-|||",coherent_b],

    ["|+||",coherent_b],
    ["|-||",coherent_b],

    ["||+|",coherent_f],
    ["||-|",coherent_f],
    
    ["|||+",coherent_f],
    ["|||-",coherent_f],

]


static_mix = [
    
    ["+|||",coherent_b],
    ["-|||",coherent_b],

    ["|+||",coherent_b],
    ["|-||",coherent_b],

    ["||+|",coherent_f],
    ["||-|",coherent_f],
    
    ["|||+",coherent_f],
    ["|||-",coherent_f],
#introducing the penalty to the Hamiltonian
    ["nn|||",penalty_quad],
    ["|nn||",penalty_quad],
    ["n|n||",penalty_quad_2],
    ["n|||",penalty_lin],
    ["|n||",penalty_lin],
    ["I|||",penalty_const]

]

static_mass = [
            ["||+|-", mass_f], # f mass term
            ["||-|+", m_mass_f], # f mass term
]

static_target = [
            ["++--|||", bosonic_part_LGT], # b^dagger*b^dagger*b*b
            ["|++--||", bosonic_part_LGT], # b^dagger*b^dagger*b*b

            ["||+|-", mass_f], # f mass term
            ["||-|+", m_mass_f], # f mass term


            ["+|-|+|-", interaction_part_LGT], #boson-fermion interaction
            ["-|+|-|+", m_interaction_part_LGT], #boson-fermion interaction
#introducing the penalty to the Hamiltonian
            ["nn|||",penalty_quad],
            ["|nn||",penalty_quad],
            ["n|n||",penalty_quad_2],
            ["n|||",penalty_lin],
            ["|n||",penalty_lin],
            ["I|||",penalty_const]

]

#Schwinger bosons at 0th link
b_0 = [[+0.5*e,0]]
d_0 = [[+0.5*e,0]]

b_0_m = [[-0.5*e,0]]
d_0_m = [[-0.5*e,0]]

#Schwinger bosons at 1st link
b_1 = [[+0.5*e,1]]
d_1 = [[+0.5*e,1]]

b_1_m = [[-0.5*e,1]]
d_1_m = [[-0.5*e,1]]

#Fermions at 0th site
rho_0 = [[-e,0]]

#Fermions at 1st site
rho_1 = [[-e,1]]

#Fermions at 2nd site
rho_2 = [[-e,2]]

#Identity at 0th site
eee_0 = [[e,0]]

#Identity at 1st site
eee_1 = [[e,1]]

#Identity at 2nd site
eee_2 = [[e,2]]


static_gauss_law_0 = [
    
    ["n|||", b_0],
    ["|n||", d_0_m],
    ["||n|", rho_0],
    ["|||n", rho_0],
    ["||I|", eee_0]

    
]



static_gauss_law_1 = [
    
    ["n|||", b_0_m],
    ["|n||", d_0],
    ["n|||", b_1],
    ["|n||", d_1_m],
    ["||n|", rho_1],
    ["|||n", rho_1],
    ["||I|", eee_1]

]    
    

static_gauss_law_2 = [
    
    ["n|||", b_1_m],
    ["|n||", d_1],
    ["||n|", rho_2],
    ["|||n", rho_2],
    ["||I|", eee_2]

    
]





################################################
###### set up Hamiltonian and initial states
no_checks = dict(check_pcon=False,check_symm=False,check_herm=False)
H_R6  = hamiltonian(bosons_hopping,dynamic,basis=basis,**no_checks)
H_R1  = hamiltonian(bosons_on_site_int,dynamic,basis=basis,**no_checks)
H_R5  = hamiltonian(fermions_hopping,dynamic,basis=basis,**no_checks)
H_R3  = hamiltonian(chem_potential_f,dynamic,basis=basis,**no_checks)
H_R4  = hamiltonian(chem_potential_b,dynamic,basis=basis,**no_checks)
H_Rb1  = hamiltonian(boson_fermion_int_b1,dynamic,basis=basis,**no_checks)
H_Rb2  = hamiltonian(boson_fermion_int_b2,dynamic,basis=basis,**no_checks)
H_Rd1  = hamiltonian(boson_fermion_int_d1,dynamic,basis=basis,**no_checks)
H_Rd2  = hamiltonian(boson_fermion_int_d2,dynamic,basis=basis,**no_checks)


Gauss_law_0 = hamiltonian(static_gauss_law_0, dynamic_list = [], basis=basis, **no_checks)
Gauss_law_1 = hamiltonian(static_gauss_law_1, dynamic_list = [], basis=basis, **no_checks)
Gauss_law_2 = hamiltonian(static_gauss_law_2, dynamic_list = [], basis=basis, **no_checks)




#H_BFM = hamiltonian(static,dynamic,basis=basis,**no_checks)
H_target = hamiltonian(static_target,dynamic,basis=basis,**no_checks)

H_mix = H_R5 + H_R6

H_mass = hamiltonian(static_mass,dynamic,basis=basis,**no_checks)

H_coh = hamiltonian(static_coh,dynamic,basis=basis,**no_checks)
H_mix = hamiltonian(static_mix,dynamic,basis=basis,**no_checks)

Gauss_quadrat_0 = Gauss_law_0*Gauss_law_0
Gauss_quadrat_1 = Gauss_law_1*Gauss_law_1
Gauss_quadrat_2 = Gauss_law_2*Gauss_law_2

reference states: 
array index   /   Fock state   /   integer repr. 
     0.         |2 2>           8  
     1.         |2 1>           7  
     2.         |2 0>           6  
     3.         |1 2>           5  
     4.         |1 1>           4  
     5.         |1 0>           3  
     6.         |0 2>           2  
     7.         |0 1>           1  
     8.         |0 0>           0  
reference states: 
array index   /   Fock state   /   integer repr. 
      0.         |1 1 1>|1 1 1>           63  
      1.         |1 1 1>|1 1 0>           62  
      2.         |1 1 1>|1 0 1>           61  
      3.         |1 1 1>|1 0 0>           60  
      4.         |1 1 1>|0 1 1>           59  
      5.         |1 1 1>|0 1 0>           58  
      6.         |1 1 1>|0 0 1>           57  
      7.         |1 1 1>|0 0 0>           56  
      8.         |1 1 0>|1 1 1>           55  
      9.         |1 1 0>|1 1 0>           54  
     10.         |1 1 0>|1 0 1>           53  
     11.         |1 1

In [2]:
static_f1f1 = [[ 10.,i,i] for i in range(L_f)]
static_f1f2 = [[ 10.,i,i] for i in range(L_f)]
static_f2f1 = [[ -10.,i,i] for i in range(L_f)]
static_f2f2 = [[ 1.,i,i] for i in range(L_f)]

static_bb = [[ 1.,i,i] for i in range(L_b)]
static_bd = [[ 1.,i,i] for i in range(L_b)]
static_db = [[ 1.,i,i] for i in range(L_b)]
static_dd = [[ 1.,i,i] for i in range(L_b)]


static_t = [

    ["||+-|",static_f1f1],
    ["||+|-",static_f1f2],
    ["||-|+",static_f2f1],
    ["|||+-",static_f2f2],

    ["+-|||",static_bb],
    ["+|-||",static_bd],
    ["-|+||",static_db],
    ["|+-||",static_dd],

]
T = hamiltonian(static_t, dynamic_list = [], basis=basis, **no_checks)

T.__init__(static_t, dynamic_list = [],basis=basis, check_herm = True)


Hermiticity check passed!


<ipython-input-2-5f4341350cb4>:27: UserWarning: Test for symmetries not implemented for <class 'quspin.basis.tensor.tensor_basis'>, to turn off this warning set check_symm=False in hamiltonian
  T.__init__(static_t, dynamic_list = [],basis=basis, check_herm = True)
<ipython-input-2-5f4341350cb4>:27: UserWarning: Test for particle conservation not implemented for <class 'quspin.basis.tensor.tensor_basis'>, to turn off this warning set check_pcon=False in hamiltonian
  T.__init__(static_t, dynamic_list = [],basis=basis, check_herm = True)


In [34]:
S = exp_op(T,a=1j)
smat = S.get_mat()
mat_arr = smat.toarray()

tmat_arr = T.toarray()


In [35]:
smartix = expm(1j*tmat_arr)

In [4]:
psi_0 = np.zeros(basis.Ns) # allocate space for state
#1.
i_0 = basis.index("21","01","101","100")
i_1 = basis.index("21","01","100","101")

#2.
i_2 = basis.index("10","12","101","001")
i_3 = basis.index("10","12","001","101")

#3.
i_4 = basis.index("22","00","110","100")
i_5 = basis.index("22","00","100","110")

#4.
i_6 = basis.index("01","21","011","010")
i_7 = basis.index("01","21","010","011")

#5.
i_8 = basis.index("12","10","110","010")
i_9 = basis.index("12","10","010","110")

#6.
i_10 = basis.index("00","22","011","001")
i_11 = basis.index("00","22","001","011")

psi_0[i_0] =1./np.sqrt(8)
psi_0[i_1] = -1./np.sqrt(8)
psi_0[i_2] = 1./np.sqrt(8)
psi_0[i_3] =-1./np.sqrt(8)
#psi_0[i_4] = 1./np.sqrt(12)
#psi_0[i_5] = -1./np.sqrt(12)
psi_0[i_6] =1./np.sqrt(8)
psi_0[i_7] = -1./np.sqrt(8)
psi_0[i_8] = 1./np.sqrt(8)
psi_0[i_9] =-1./np.sqrt(8)
#psi_0[i_10] = 1./np.sqrt(12)
#psi_0[i_11] = -1./np.sqrt(12)

print(np.dot(psi_0,psi_0))
print(Gauss_law_0.expt_value(psi_0))
print(Gauss_law_1.expt_value(psi_0))
print(Gauss_law_2.expt_value(psi_0))
print(H_target.expt_value(psi_0))

0.9999999999999998
0j
0j
0j
(10.35000000000018+0j)


In [8]:
psi = mat_arr@psi_0
print(psi_0)
print(psi)
print(np.dot(psi_0,psi_0))
print(np.dot(psi,psi))

[0. 0. 0. ... 0. 0. 0.]
[0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
0.9999999999999998
(-0.11253895381322072-0.011869704228012373j)


In [73]:
def is_unitary(m):
    return np.allclose(np.eye(basis.Ns), m.conj().T @ m, rtol=1e-1, atol=1e-1)


In [74]:
print(is_unitary(smatrix))

False


In [37]:
def check_hermitian(a, rtol=1e-10, atol=1e-10):
    return np.allclose(a, a.conj().T, rtol=rtol, atol=atol)

In [38]:
print(check_hermitian(tmat_arr))

True


In [52]:
eigv, eigvec = H_target.eigh()

In [53]:
psi_0 = eigvec[:,0]

In [75]:
def extract_state(theta,psi):
    psi_t0 = H_R1.evolve(psi,0.0,np.array([theta[0]]))
    psi_t1 = H_Rb1.evolve(psi_t0,0.0,np.array([theta[1]]))
    psi_t2 = H_Rb2.evolve(psi_t1,0.0,np.array([theta[2]]))
    psi_t3 = H_Rd1.evolve(psi_t2,0.0,np.array([theta[3]]))
    psi_t4 = H_Rd2.evolve(psi_t3,0.0,np.array([theta[4]]))
    psi_t5 = H_R5.evolve(psi_t4,0.0,np.array([theta[5]]))
    psi_t6 = H_R6.evolve(psi_t5,0.0,2*np.array([theta[6]]))
    psi_t7 = H_R5.evolve(psi_t6,0.0,np.array([theta[5]]))
    psi_t8 = H_Rb1.evolve(psi_t7,0.0,np.array([theta[1]]))
    psi_t9 = H_Rb2.evolve(psi_t8,0.0,np.array([theta[2]]))
    psi_t10 = H_Rd1.evolve(psi_t9,0.0,np.array([theta[3]]))
    psi_t11 = H_Rd2.evolve(psi_t10,0.0,np.array([theta[4]]))
    psi_t12 = H_R1.evolve(psi_t11,0.0,np.array([theta[0]]))
    
    psi_final = psi_t12
    
    return psi_final

In [78]:
def extract_state(theta,psi):
    psi_t0 = H_R1.evolve(psi,0.0,theta[0])
    psi_t1 = H_Rb1.evolve(psi_t0,0.0,theta[1])
    psi_t2 = H_Rb2.evolve(psi_t1,0.0,theta[2])
    psi_t3 = H_Rd1.evolve(psi_t2,0.0,theta[3])
    psi_t4 = H_Rd2.evolve(psi_t3,0.0,theta[4])
    psi_t5 = H_R5.evolve(psi_t4,0.0,theta[5])
    psi_t6 = H_R6.evolve(psi_t5,0.0,2*theta[6])
    psi_t7 = H_R5.evolve(psi_t6,0.0,theta[5])
    psi_t8 = H_Rb1.evolve(psi_t7,0.0,theta[1])
    psi_t9 = H_Rb2.evolve(psi_t8,0.0,theta[2])
    psi_t10 = H_Rd1.evolve(psi_t9,0.0,theta[3])
    psi_t11 = H_Rd2.evolve(psi_t10,0.0,theta[4])
    psi_t12 = H_R1.evolve(psi_t11,0.0,theta[0])
    psi_final = psi_t12
    
    return psi_final

In [80]:
ar = np.array([1.,0.,0.,0.,0.,0.,1.])
a = extract_state(ar,psi_0)
print(np.dot(a,a))

(0.5650890149389957+0.2667337908449309j)


In [101]:
psi_1 = H_R1.evolve(psi_0,0.0,np.array([1.,2.,3.]),verbose = True)
state = np.array([row[0] for row in psi_1])
print(np.dot(state, state))

evolved to time 1.0, norm of state(s) 0.9999999996274187
evolved to time 2.0, norm of state(s) 0.9999999989012816
evolved to time 3.0, norm of state(s) 0.9999999985364019
(0.704029612624521-0.1496928462010768j)
